In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RDD Example").getOrCreate()
sc = spark.sparkContext

24/12/10 13:32:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 13:32:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
movies_rdd = sc.parallelize([
    (1, ('어벤져스', '마블')),
    (2, ('아이언맨', '마블')),
    (3, ('배트맨', 'DC')),
    (4, ('슈퍼맨', 'DC')),
    (5, ('토르', '마블')),
    (6, ('스파이더맨', '마블')),
    (7, ('블랙팬서', '마블')),
    (8, ('저스티스 리그', 'DC'))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, 'KR')),
    (2, (9238472, 'US')),
    (3, (8432921, 'JP')),
    (4, (2348921, 'UK')),
    (5, (1238476, 'IN')),
    (6, (7432198, 'DE')),
    (7, (5673849, 'FR')),
    (8, (9874321, 'BR'))
])

## 관객 수가 500만 이상인 영화를 가져오기
- 필터링한 뒤 조인을 하는 것이 더 효율적

In [3]:
# 조인 먼저
movies_att = movies_rdd.join(attendances_rdd)
movies_att.filter(lambda x:x[1][1][0] >= 5000000).collect()

[(8, (('저스티스 리그', 'DC'), (9874321, 'BR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (2, (('아이언맨', '마블'), (9238472, 'US'))),
 (6, (('스파이더맨', '마블'), (7432198, 'DE'))),
 (3, (('배트맨', 'DC'), (8432921, 'JP'))),
 (7, (('블랙팬서', '마블'), (5673849, 'FR')))]

In [4]:
# 필터 먼저
filtered_att = attendances_rdd.filter(lambda x:x[1][0] >= 5000000)
movies_rdd.join(filtered_att).collect()

[(8, (('저스티스 리그', 'DC'), (9874321, 'BR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (2, (('아이언맨', '마블'), (9238472, 'US'))),
 (6, (('스파이더맨', '마블'), (7432198, 'DE'))),
 (3, (('배트맨', 'DC'), (8432921, 'JP'))),
 (7, (('블랙팬서', '마블'), (5673849, 'FR')))]

In [6]:
movies = [
    (1, '어벤져스', '마블', 2019, 4, 25),
    (2, '아이언맨', '마블', 2008, 5, 2),
    (3, '배트맨', 'DC', 2005, 6, 15),
    (4, '슈퍼맨', 'DC', 1978, 12, 15),
    (5, '토르', '마블', 2011, 4, 10),
    (6, '스파이더맨', '마블', 2002, 7, 3),
    (7, '블랙팬서', '마블', 2018, 2, 16),
    (8, '저스티스 리그', 'DC', 2017, 11, 17)
]

movie_schema = ['ID', 'Title', 'Genre', 'Year', 'Month', 'Day']

df = spark.createDataFrame(data=movies, schema=movie_schema)
df.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)



In [7]:
df.show()

+---+-------------+-----+----+-----+---+
| ID|        Title|Genre|Year|Month|Day|
+---+-------------+-----+----+-----+---+
|  1|     어벤져스| 마블|2019|    4| 25|
|  2|     아이언맨| 마블|2008|    5|  2|
|  3|       배트맨|   DC|2005|    6| 15|
|  4|       슈퍼맨|   DC|1978|   12| 15|
|  5|         토르| 마블|2011|    4| 10|
|  6|   스파이더맨| 마블|2002|    7|  3|
|  7|     블랙팬서| 마블|2018|    2| 16|
|  8|저스티스 리그|   DC|2017|   11| 17|
+---+-------------+-----+----+-----+---+



In [9]:
df.select('ID').show()

+---+
| ID|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
+---+



In [10]:
df.filter(df.Year >=2010).show()

+---+-------------+-----+----+-----+---+
| ID|        Title|Genre|Year|Month|Day|
+---+-------------+-----+----+-----+---+
|  1|     어벤져스| 마블|2019|    4| 25|
|  5|         토르| 마블|2011|    4| 10|
|  7|     블랙팬서| 마블|2018|    2| 16|
|  8|저스티스 리그|   DC|2017|   11| 17|
+---+-------------+-----+----+-----+---+



## View 등록

In [11]:
df.createOrReplaceTempView('movies')

In [12]:
query = """
select * from movies
"""

spark.sql(query).show()

+---+-------------+-----+----+-----+---+
| ID|        Title|Genre|Year|Month|Day|
+---+-------------+-----+----+-----+---+
|  1|     어벤져스| 마블|2019|    4| 25|
|  2|     아이언맨| 마블|2008|    5|  2|
|  3|       배트맨|   DC|2005|    6| 15|
|  4|       슈퍼맨|   DC|1978|   12| 15|
|  5|         토르| 마블|2011|    4| 10|
|  6|   스파이더맨| 마블|2002|    7|  3|
|  7|     블랙팬서| 마블|2018|    2| 16|
|  8|저스티스 리그|   DC|2017|   11| 17|
+---+-------------+-----+----+-----+---+



## 2010년 이후 개봉한 영화

In [13]:
query = '''
select ID, Title
from movies
where Year > 2010
'''

spark.sql(query).show()

+---+-------------+
| ID|        Title|
+---+-------------+
|  1|     어벤져스|
|  5|         토르|
|  7|     블랙팬서|
|  8|저스티스 리그|
+---+-------------+



## 장르가 마블인 영화 

In [15]:
query = '''
select Title
from movies
where Genre = '마블'
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  어벤져스|
|  아이언맨|
|      토르|
|스파이더맨|
|  블랙팬서|
+----------+



## 제목이 '맨'으로 끝나는 영화

In [16]:
query = '''
select Title
from movies
where Title Like "%맨"
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  아이언맨|
|    배트맨|
|    슈퍼맨|
|스파이더맨|
+----------+



## 제목에 '이'가 들어가 있는 영화 

In [18]:
query = '''
select Title
from movies
where Title Like "%이%"
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  아이언맨|
|스파이더맨|
+----------+



## 개봉월이 4~8월 사이인 영화

In [19]:
query = '''
select Title
from movies
where Month Between 4 And 8
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  어벤져스|
|  아이언맨|
|    배트맨|
|      토르|
|스파이더맨|
+----------+



## 제목이 '맨'으로 끝나면서 개봉연도가 2000년 이상인 영화

In [21]:
query = '''
select Title
from movies
where Year >= 2000 And Title Like '%맨'
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  아이언맨|
|    배트맨|
|스파이더맨|
+----------+



## 장르가 마블, DC인 영화

In [22]:
query = '''
select Title
from movies
where Genre in ('마블','DC')
'''

spark.sql(query).show()

+-------------+
|        Title|
+-------------+
|     어벤져스|
|     아이언맨|
|       배트맨|
|       슈퍼맨|
|         토르|
|   스파이더맨|
|     블랙팬서|
|저스티스 리그|
+-------------+



## 장르가 '마'로 시작하거나 제목이 '맨'으로 끝나는 영화 중 2000년 이후로 개봉한 영화

In [23]:
query = '''
select Title
from movies
where Genre Like '%마' or (Title Like '%맨' And Year > 2000)
'''

spark.sql(query).show()

+----------+
|     Title|
+----------+
|  아이언맨|
|    배트맨|
|스파이더맨|
+----------+



## count()

In [28]:
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType, StringType

attendances = [
    (1, 13934592, 'KR'),
    (2, 9238472, 'US'),
    (3, 8432921, 'JP'),
    (4, 2348921, 'UK'),
    (5, 1238476, 'IN'),
    (6, 7432198, 'DE'),
    (7, 5673849, 'FR'),
    (8, 9874321, 'BR')
]

att_schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('att', IntegerType(), True),
    StructField('theater_country', StringType(), True)
])

att_df = spark.createDataFrame(data=attendances, schema=att_schema)
att_df.dtypes

[('id', 'int'), ('att', 'int'), ('theater_country', 'string')]

In [29]:
att_df.show()

+---+--------+---------------+
| id|     att|theater_country|
+---+--------+---------------+
|  1|13934592|             KR|
|  2| 9238472|             US|
|  3| 8432921|             JP|
|  4| 2348921|             UK|
|  5| 1238476|             IN|
|  6| 7432198|             DE|
|  7| 5673849|             FR|
|  8| 9874321|             BR|
+---+--------+---------------+



In [30]:
att_df.createOrReplaceTempView('att')

In [31]:
att_df.select('*').show()

+---+--------+---------------+
| id|     att|theater_country|
+---+--------+---------------+
|  1|13934592|             KR|
|  2| 9238472|             US|
|  3| 8432921|             JP|
|  4| 2348921|             UK|
|  5| 1238476|             IN|
|  6| 7432198|             DE|
|  7| 5673849|             FR|
|  8| 9874321|             BR|
+---+--------+---------------+



In [32]:
query = '''
select *
from movies
join att 
on movies.id = att.id
'''

spark.sql(query).show()

+---+-------------+-----+----+-----+---+---+--------+---------------+
| ID|        Title|Genre|Year|Month|Day| id|     att|theater_country|
+---+-------------+-----+----+-----+---+---+--------+---------------+
|  7|     블랙팬서| 마블|2018|    2| 16|  7| 5673849|             FR|
|  6|   스파이더맨| 마블|2002|    7|  3|  6| 7432198|             DE|
|  5|         토르| 마블|2011|    4| 10|  5| 1238476|             IN|
|  1|     어벤져스| 마블|2019|    4| 25|  1|13934592|             KR|
|  3|       배트맨|   DC|2005|    6| 15|  3| 8432921|             JP|
|  8|저스티스 리그|   DC|2017|   11| 17|  8| 9874321|             BR|
|  2|     아이언맨| 마블|2008|    5|  2|  2| 9238472|             US|
|  4|       슈퍼맨|   DC|1978|   12| 15|  4| 2348921|             UK|
+---+-------------+-----+----+-----+---+---+--------+---------------+



In [34]:
query = '''
select movies.ID, movies.Title, movies.Genre, att.att, att.theater_country
from movies
join att 
on movies.id = att.id
'''

spark.sql(query).show()

+---+-------------+-----+--------+---------------+
| ID|        Title|Genre|     att|theater_country|
+---+-------------+-----+--------+---------------+
|  7|     블랙팬서| 마블| 5673849|             FR|
|  6|   스파이더맨| 마블| 7432198|             DE|
|  5|         토르| 마블| 1238476|             IN|
|  1|     어벤져스| 마블|13934592|             KR|
|  3|       배트맨|   DC| 8432921|             JP|
|  8|저스티스 리그|   DC| 9874321|             BR|
|  2|     아이언맨| 마블| 9238472|             US|
|  4|       슈퍼맨|   DC| 2348921|             UK|
+---+-------------+-----+--------+---------------+



In [35]:
movies_views = spark.sql(query)
movies_views.createOrReplaceTempView('movies_view')

In [38]:
query = '''
select * 
from movies_view
'''

spark.sql(query).show()

+---+-------------+-----+--------+---------------+
| ID|        Title|Genre|     att|theater_country|
+---+-------------+-----+--------+---------------+
|  7|     블랙팬서| 마블| 5673849|             FR|
|  6|   스파이더맨| 마블| 7432198|             DE|
|  5|         토르| 마블| 1238476|             IN|
|  1|     어벤져스| 마블|13934592|             KR|
|  3|       배트맨|   DC| 8432921|             JP|
|  8|저스티스 리그|   DC| 9874321|             BR|
|  2|     아이언맨| 마블| 9238472|             US|
|  4|       슈퍼맨|   DC| 2348921|             UK|
+---+-------------+-----+--------+---------------+



In [39]:
spark.stop()